In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ibm_db

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 MB 9.2 MB/s eta 0:00:00


In [3]:
import ibm_db
import os
import pandas as pd

# IBM DB2 connection setup
dsn_hostname = os.getenv('DB_HOST', '0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud') # change this
dsn_uid = os.getenv('DB_UID', 'cyp22649')  # change this
dsn_pwd = os.getenv('DB_PWD', '5CoauM9zBgPEGap9')  # change this
dsn_port = os.getenv('DB_PORT', '31198')  # change this
dsn_database = "bludb"  # change if necessary
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_protocol = "TCPIP"
dsn_security = "SSL"


# Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd, dsn_security)

# Create database connection
try:
    conn = ibm_db.connect(dsn, "", "")
    print("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

    # Query to get the first 5 rows from the table
    command = "SELECT * FROM CB106 FETCH FIRST 1 ROWS ONLY"   # Câu lệnh SQL command = "SELECT * FROM cb106 FETCH FIRST 1 ROWS ONLY" đơn giản là truy vấn để lấy một dòng dữ liệu đầu tiên từ bảng cb106 trong cơ sở dữ liệu.
                                                              # Đây là một cú pháp phổ biến trong SQL để giới hạn số lượng hàng trả về.
    # SELECT column_name FROM cb106 FETCH FIRST 1 ROWS ONLY;  # Lấy cột column_name hàng 1

    stmt = ibm_db.exec_immediate(conn, command)

    # Fetch the result set
    result = ibm_db.fetch_both(stmt)
    rows = []

    # Loop through the result and fetch the rows
    while result:
        rows.append(result)
        result = ibm_db.fetch_both(stmt)

    # Convert result to a pandas DataFrame for better readability
    df = pd.DataFrame(rows)
    print(df)

except Exception as e:
    print("Error connecting to the database: ", str(e))
finally:
    # Close the connection
    if conn:
        ibm_db.close(conn)


Connected to database:  bludb as user:  cyp22649 on host:  0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud
  Tên Sinh Viên             0   Ngày Sinh           1  Mã Số      2  \
0  Nguyễn Văn A  Nguyễn Văn A  1999-01-15  1999-01-15  SV001  SV001   

          Chuyên Ngành                    3  
0  Công nghệ thông tin  Công nghệ thông tin  


In [8]:
from google.colab.output import eval_js
print(eval_js('google.colab.kernel.proxyPort(5000)'))

https://zyn4hdjdyu-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [5]:
def fetch_students():
    try:
        conn = ibm_db.connect(dsn, "", "")
        query = "SELECT * FROM CB106"
        stmt = ibm_db.exec_immediate(conn, query)
        rows = []
        result = ibm_db.fetch_assoc(stmt)
        while result:
            rows.append(result)
            result = ibm_db.fetch_assoc(stmt)
        df = pd.DataFrame(rows)
        return df
    except Exception as e:
        print("Error fetching students from DB2: ", str(e))
        return None
    finally:
        if 'conn' in locals():
            ibm_db.close(conn)

# Add a student to DB2
def add_student(name, birth_date, student_id, major):
    try:
        conn = ibm_db.connect(dsn, "", "")
        query = """
        INSERT INTO CB106 ("Tên Sinh Viên", "Ngày Sinh", "Mã Số", "Chuyên Ngành")
        VALUES (?, ?, ?, ?)
        """
        stmt = ibm_db.prepare(conn, query)
        ibm_db.bind_param(stmt, 1, name)
        ibm_db.bind_param(stmt, 2, birth_date)
        ibm_db.bind_param(stmt, 3, student_id)
        ibm_db.bind_param(stmt, 4, major)
        ibm_db.execute(stmt)
    except Exception as e:
        print("Error adding student to DB2: ", str(e))
    finally:
        if 'conn' in locals():
            ibm_db.close(conn)


# Delete a student from DB2
def delete_student(student_id):
    try:
        conn = ibm_db.connect(dsn, "", "")
        query = "DELETE FROM CB106 WHERE \"Mã Số\" = ?"
        stmt = ibm_db.prepare(conn, query)
        ibm_db.bind_param(stmt, 1, student_id)
        ibm_db.execute(stmt)
    except Exception as e:
        print("Error deleting student from DB2: ", str(e))
    finally:
        if 'conn' in locals():
            ibm_db.close(conn)

# Update a student's information in DB2
def update_student(student_id, name=None, birth_date=None, major=None):
    try:
        conn = ibm_db.connect(dsn, "", "")
        # Base query to update the student information
        query = "UPDATE CB106 SET"

        updates = []
        params = []

        # Check if each field is provided and add it to the query
        if name is not None:
            updates.append("\"Tên Sinh Viên\" = ?")
            params.append(name)
        if birth_date is not None:
            updates.append("\"Ngày Sinh\" = ?")
            params.append(birth_date)
        if major is not None:
            updates.append("\"Chuyên Ngành\" = ?")
            params.append(major)

        # Join the updates with commas and add the WHERE condition
        if updates:
            query += " " + ", ".join(updates) + " WHERE \"Mã Số\" = ?"
            params.append(student_id)

            stmt = ibm_db.prepare(conn, query)

            # Bind parameters dynamically
            for i, param in enumerate(params):
                ibm_db.bind_param(stmt, i + 1, param)

            ibm_db.execute(stmt)
            print("Student information updated successfully.")
        else:
            print("No fields provided to update.")
    except Exception as e:
        print("Error updating student in DB2: ", str(e))
    finally:
        if 'conn' in locals():
            ibm_db.close(conn)



In [6]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [9]:
from flask import Flask, render_template, request, redirect, url_for
import ibm_db
import pandas as pd
import os


app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        if 'add' in request.form:
            name = request.form['name']
            birth_date = request.form['birth_date']
            student_id = request.form['student_id']
            major = request.form['major']
            # Add student to DB2
            add_student(name, birth_date, student_id, major)
        elif 'delete' in request.form:
            student_id = request.form['student_id']
            # Delete student from DB2
            delete_student(student_id)
        elif 'update' in request.form:
            # Handle updating student
            student_id = request.form['student_id']
            name = request.form.get('name')
            birth_date = request.form.get('birth_date')
            major = request.form.get('major')
            update_student(student_id, name, birth_date, major)
        # After updating DB2, redirect to refresh the page with updated student list
        # return redirect(url_for('index'))

    # Fetch updated student list from DB2 and display
    students = fetch_students()
    return render_template('index.html', students=students)
    # return

# Run the Flask app
if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 13:26:06] "GET /?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 13:26:07] "GET /favicon.ico?authuser=0 HTTP/1.1" 404 -
